<a href="https://colab.research.google.com/github/SumruNayir/Tokenizer/blob/master/tokenize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas as pd
import networkx as nx
from sklearn import preprocessing 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


In [2]:
# Load the data
from google.colab import files # Use to load data on Google Colab
uploaded = files.upload() # Use to load data on Google Colab

Saving deceptive-opinion.csv to deceptive-opinion.csv


Pozitif ve Negatiflerin olduğu kolon adı Polarity. Ve buralarda işlem yapmak için bu bu kolon adının kullanılması gerekiyor.

In [6]:
datasetOrijinal = pd.read_csv("deceptive-opinion.csv")
dataset = pd.read_csv("deceptive-opinion.csv")
dataset['polarity'].unique() 

array(['positive', 'negative'], dtype=object)

In [14]:
label_encoder = preprocessing.LabelEncoder() 
dataset['polarity']= label_encoder.fit_transform(dataset['polarity']) 
dataset['polarity'].unique() 
dataset['polarity']

0       1
1       1
2       1
3       1
4       1
       ..
1595    0
1596    0
1597    0
1598    0
1599    0
Name: polarity, Length: 1600, dtype: int64

In [0]:
target = dataset['polarity'].values.tolist()
data = dataset['text'].values.tolist()

In [12]:
datasetOrijinal

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...
...,...,...,...,...,...
1595,deceptive,intercontinental,negative,MTurk,Problems started when I booked the InterContin...
1596,deceptive,amalfi,negative,MTurk,The Amalfi Hotel has a beautiful website and i...
1597,deceptive,intercontinental,negative,MTurk,The Intercontinental Chicago Magnificent Mile ...
1598,deceptive,palmer,negative,MTurk,"The Palmer House Hilton, while it looks good i..."


Pozitifler 1, negatifler 0 yapıldı.

In [13]:
dataset

,deceptive,hotel,polarity,source,text
0,truthful,conrad,1,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,1,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,1,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,1,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,1,TripAdvisor,I asked for a high floor away from the elevato...
...,...,...,...,...,...
1595,deceptive,intercontinental,0,MTurk,Problems started when I booked the InterContin...
1596,deceptive,amalfi,0,MTurk,The Amalfi Hotel has a beautiful website and i...
1597,deceptive,intercontinental,0,MTurk,The Intercontinental Chicago Magnificent Mile ...
1598,deceptive,palmer,0,MTurk,"The Palmer House Hilton, while it looks good i..."


Orijinaller ve testler için datalar oluşturuldu ve çıkan cümlenin pozitifliği-negatifliği kontrol edildi.

In [0]:
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:] #text
y_train, y_test = target[:cutoff], target[cutoff:] #

In [18]:
x_train[100]

'i stayed at this hotel for a week with my family this hotel is huge so clean has comfy beds foods great staff couldn,t of being any nicer. stayed here for thanksgiving it was fabulous second time in chicago and still want to go back again ... the hotel is within walkin distance to both state street and michicgan avenue.There,s taxis right outside the hotel and the concierge can also ring for mini buses if your travelling with a big crowd.there,s also a cinema 2 mins away opposite the hotel . Perfect Hotel for a great get away ... \n'

In [19]:
y_train[100]

1

In [0]:
num_words = 1000
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(data)

In [0]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

Tokenizer işlemi yapıldı.

In [23]:
x_train_tokens[100]

[5,
 47,
 14,
 22,
 9,
 11,
 4,
 354,
 18,
 15,
 246,
 22,
 9,
 16,
 339,
 39,
 76,
 108,
 774,
 159,
 40,
 41,
 8,
 197,
 126,
 948,
 47,
 75,
 11,
 13,
 6,
 646,
 388,
 59,
 7,
 24,
 2,
 247,
 200,
 3,
 104,
 81,
 62,
 1,
 9,
 16,
 367,
 343,
 3,
 271,
 973,
 314,
 2,
 541,
 31,
 154,
 398,
 1,
 9,
 2,
 1,
 242,
 119,
 74,
 11,
 635,
 54,
 113,
 18,
 4,
 303,
 31,
 74,
 4,
 129,
 166,
 1,
 9,
 237,
 9,
 11,
 4,
 40,
 56,
 166]

In [0]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

Data'da bulunan tğm cümler için ortak token sayısı

In [25]:
average = np.mean(num_tokens)
average

130.435

In [26]:
counter = np.max(num_tokens)
counter

676

En çok token bulunan yorumun indexi

In [27]:
index = np.argmax(num_tokens)
index

1473

Yorumlar için ortalama token sayısı

In [29]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
np.sum(num_tokens < max_tokens) / len(num_tokens)
max_tokens

279

In [0]:
x_train_pad =pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [32]:
np.array(x_train_tokens[800])

array([148,   4,  12,   4, 720,  72,  11, 129, 173,  11,   4, 174,   2,
         6, 485, 249,  72,   6, 483,   1, 720,  63,   6, 357,   4, 472,
        19,   3, 717,  13,   6,  85,   8,   1, 310,   7,   1,  91,  21,
        88, 886,   2, 332, 626,   8, 298, 218,   3, 302,   1,  34,  10,
        71, 421,  94,  46, 485, 882,  36,   1, 279,   6, 151,  31,   6,
         4,  28, 225, 298,  17,  10,  71,  19,  56,   8, 951,  10,   1,
       279, 397, 169,   6,  28,   7,   1, 210,  13,  16,   7,   4,  40,
        65, 231,  92,  59,   5,  57, 405,  45,   8,   1,  94, 103,   7,
         1, 123,  55,   1, 417,  63,  22, 534,  19,  33, 156, 230,  54,
        32,  44, 201, 744,  23, 107,  25,   5, 493, 106,  13,  11,   4,
       836,  53,  63,  96])

Bulunan token sayısına göre eksik olan kısımlar 0 ile dolduruldu

In [33]:
x_train_pad[800]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 148,   4,  12,   4, 720,  72,  11, 129, 173,  11,   4,
       174,   2,   6, 485, 249,  72,   6, 483,   1, 720,  63,   6, 357,
         4, 472,  19,   3, 717,  13,   6,  85,   8,   1, 310,   